# Neighborhoods in Toronto
This notebook explores neighborhoods in Toronto based on postal code.

Importing necessary libraries ...

In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Neighborhoods and postal codes from common Wikipedia link

In [13]:
# Finding neighborhoods and postal codes from common Wikipedia link and creating a dataframe
import requests
from bs4 import BeautifulSoup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')

table = soup.find_all('table')[0] 

column_names = ['Postcode', 'Borough', 'Neighborhood'] 
    
new_table = pd.DataFrame(columns=column_names) 

# Remove Not Assigned Boroughs and Neighborhoods
row_marker = 0

for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        if columns != [] and not 'Not assigned' in columns[1]:
            for column in columns:
                if column_marker == 2 and 'Not assigned' in column.get_text():
                    new_table.loc[row_marker,column_names[column_marker]] = columns[column_marker-1].get_text()
                else:
                    new_table.loc[row_marker,column_names[column_marker]] = column.get_text().replace("\n","")
                column_marker += 1
            row_marker += 1
# show new table
new_table.head()
    




,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [14]:
# Concatenate same Neighborhoods into one Borough
new_table['Neighborhood'] = new_table['Neighborhood'] + ", "
final_table = new_table.groupby(['Postcode', 'Borough'], as_index=False).agg(sum)
final_table['Neighborhood'].replace(r', $', '',inplace=True, regex=True)
final_table.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
# Size of new table
final_table.shape

(103, 3)

In [16]:
# tried geocoder, but it doesn't seem to work
#import geocoder 
#
#for row in final_table.itertuples(index=True, name='Pandas'):
#    print (row)
#
#    # initialize your variable to None
#    lat_lng_coords = None
#    print (lat_lng_coords)
#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#      print("ennen")
#      g = geocoder.google('{}, Toronto, Ontario'.format(getattr(row, "Postcode")))
#      g = geocoder.google('Toronto, Ontario')
#      print ("jalkeen")  
#      lat_lng_coords = g.latlng
#      print (g.latlng)
#
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]
#    final_table.insert(3, "Latitude", latitude) 
#    final_table_table.insert(4, "Longitude", longitude)                       
    
#Pandas.head()

### Including spatial data

In [17]:
# Getting corresponding postal codes
!wget -q -O 'Postal_codes.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Postalcodes_df = pd.read_csv('Postal_codes.csv')
Postalcodes_df.rename(columns={'Postal Code':'Postcode'}, inplace=True)
 
Postalcodes_df.head()

Data downloaded!


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# Merge dataframes
complete_df= pd.merge(final_table, Postalcodes_df, on='Postcode', how='outer')
complete_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
# Getting Toronto location
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Showing neighborhoods on map

In [21]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(complete_df['Latitude'], complete_df['Longitude'], complete_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto